this is the notebook where I will augment my data extract my Features,and decide what architecture to use and train my model

In [1]:

import numpy as np
import pandas as pd
import torch
import torchaudio
import warnings
import os
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import OneCycleLR
from torchsummary import summary
from tqdm import tqdm
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

import librosa
from torchaudio.functional import preemphasis

import torch.nn.functional as F
import random
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

In [2]:
import os
os.getcwd()

'c:\\CNN Project'

In [3]:
df=pd.read_csv("dataset2.csv")
df.head()

,path,chapter,reciter
0,abd_albassit_abd_samad_1_rir.wav,surah_al_dhoha,abd_albassit_abd_samad
1,abd_albassit_abd_samad_1_org.wav,surah_al_dhoha,abd_albassit_abd_samad
2,abd_albassit_abd_samad_2_bg.wav,surah_al_dhoha,abd_albassit_abd_samad
3,abd_albassit_abd_samad_2_org.wav,surah_al_dhoha,abd_albassit_abd_samad
4,abd_albassit_abd_samad_3_bg.wav,surah_al_dhoha,abd_albassit_abd_samad


In [2]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

#Data Augmentation

my possible options:-Noise Injection/-Reverberation /- Background Noise Simulation/-Volume Perturbation/-Clipping/-Equalization (EQ) Changes

##Loading Data Done


In [ ]:
class OptimizedAudioDataset(Dataset):
    def __init__(self, csv_file, audio_dir, transform=None, max_length=22050*5,is_test=False):#max length change 
        self.data = pd.read_csv(csv_file)
        self.audio_dir = audio_dir
        self.transform = transform
        self.max_length = max_length
        if not is_test:
            self.label_encoder = LabelEncoder()
            self.data['reciter'] = self.label_encoder.fit_transform(self.data['reciter'])
        self.is_test = is_test

        # Pre-load all audio files
        self.audio_data = self.preload_audio(self.max_length)

    def preload_audio(self,max_length):
        audio_data = {}
        for _, row in tqdm(self.data.iterrows(), total=len(self.data), desc="Preloading audio"):
            audio_path = os.path.join(self.audio_dir, row['path'])
            waveform, sample_rate = torchaudio.load(audio_path)
            if sample_rate != 16000:
                # Resample the audio
                resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
                waveform = resampler(waveform)
            if waveform.shape[0] > 1:
                waveform = torch.mean(waveform, dim=0, keepdim=True)
            # if waveform.shape[1] < self.max_length:
            #     padding = torch.zeros(1, self.max_length - waveform.shape[1])
            #     waveform = torch.cat([waveform, padding], dim=1)
            # else:
            #     waveform = waveform[:, :self.max_length]
            if waveform.shape[-1] < max_length:
                padding = torch.zeros(max_length - waveform.shape[-1])
                padded_audio = torch.cat((waveform, padding), dim=-1)
                audio_data[row['path']] = padded_audio
            elif waveform.shape[-1] > max_length:
                truncated_audio = waveform[:, :max_length]
                audio_data[row['path']] = truncated_audio
            else:
                audio_data[row['path']] = waveform
        return audio_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        
        audio_filepath = self.data.iloc[idx]['path']
        waveform = self.audio_data[audio_filepath]

        if self.transform:
            waveform = self.transform(waveform)

        
        if self.is_test:
            return waveform
        else:
            label = self.data.iloc[idx]['reciter']
            return waveform, label
       

def optimized_audio_transform(waveform, sample_rate=16000, n_mfcc=30, n_fft=400, hop_length=120):
    # Compute MFCC
    mfcc_transform = torchaudio.transforms.MFCC(
        sample_rate=sample_rate,
        n_mfcc=n_mfcc,
        melkwargs={
            "n_fft": n_fft,
            "hop_length": hop_length,
            "n_mels": 128,
            "center": False
        }
    )
    mfcc = mfcc_transform(waveform)  # Shape: (batch, n_mfcc, timesteps)


    # Reshape to (timesteps, features)
    features = mfcc.squeeze(0).transpose(0, 1)  # Shape: (timesteps, features)

    return features # Reshape to (time_steps, n_lfcc)

def custom_collate_fn(batch):
    if isinstance(batch[0], tuple):
        waveforms, labels = zip(*batch)
        waveforms = torch.stack(waveforms)
        return waveforms, torch.tensor(labels)
    else:
        return torch.stack(batch)

In [6]:


train_dataset = OptimizedAudioDataset(csv_file="dataset2.csv",
                                      audio_dir="Dataset",
                                      transform=optimized_audio_transform)

train_data, val_data = train_test_split(train_dataset, test_size=0.1, random_state=42,
                                        stratify=train_dataset.data['reciter'])

train_loader = DataLoader(train_data, batch_size=100, shuffle=True,
                          collate_fn=custom_collate_fn, num_workers=0)

val_loader = DataLoader(val_data, batch_size=100, shuffle=False,
                        collate_fn=custom_collate_fn, num_workers=0)

# test_dataset = OptimizedAudioDataset(csv_file="/content/Test_1 (1).csv",
#                                      audio_dir="/content/drive/MyDrive/Cleaned_TechCabal_01",
#                                      transform=optimized_audio_transform,
#                                      is_test=True)

# test_loader = DataLoader(test_dataset, batch_size=70, shuffle=False,
#                          collate_fn=custom_collate_fn, num_workers=4)

Preloading audio: 100%|██████████| 6650/6650 [01:18<00:00, 84.90it/s]


In [ ]:
t=[]
for i in range (1000):
    data,label=train_data[i]
    t.append(data.shape[0])

In [7]:
from collections import Counter

Counter(t)

Counter({690: 1000})

##Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class SOrthConv(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, padding_mode='zeros'):
        '''
        Conv1d with a method for stepping towards semi-orthongonality
        http://danielpovey.com/files/2018_interspeech_tdnnf.pdf
        '''
        super(SOrthConv, self).__init__()

        kwargs = {'bias': False}
        self.conv = nn.Conv1d(in_channels, out_channels,
                              kernel_size, stride=stride,
                              padding=padding, dilation=dilation,
                              bias=False, padding_mode=padding_mode)
       
        self.reset_parameters()
        #alternative for initializeing parameters 
        # nn.init.kaiming_normal_(self.conv.weight, nonlinearity='relu')


    def forward(self, x):
        x = self.conv(x)
        return x

    def step_semi_orth(self):
        with torch.no_grad():
            M = self.get_semi_orth_weight(self.conv)
            self.conv.weight.copy_(M)

    def reset_parameters(self):
        # Standard dev of M init values is inverse of sqrt of num cols
        # Glorot-style initialization
        nn.init._no_grad_normal_(self.conv.weight, 0.,
                                 self.get_M_shape(self.conv.weight)[1]**-0.5)

    def orth_error(self):
        return self.get_semi_orth_error(self.conv).item()

    @staticmethod
    def get_semi_orth_weight(conv1dlayer):
        # updates conv1 weight M using update rule to make it more semi orthogonal
        # based off ConstrainOrthonormalInternal in nnet-utils.cc in Kaldi src/nnet3
        # includes the tweaks related to slowing the update speed
        # only an implementation of the 'floating scale' case
        with torch.no_grad():
            update_speed = 0.125 #1/8
            orig_shape = conv1dlayer.weight.shape
            # a conv weight differs slightly from TDNN formulation:
            # Conv weight: (out_filters, in_filters, kernel_width)
            # TDNN weight M is of shape: (in_dim, out_dim) or [rows, cols]
            # the in_dim of the TDNN weight is equivalent to in_filters * kernel_width of the Conv
            M = conv1dlayer.weight.reshape(
                orig_shape[0], orig_shape[1]*orig_shape[2]).T
            # M now has shape (in_dim[rows], out_dim[cols])
            mshape = M.shape
            if mshape[0] > mshape[1]:  # semi orthogonal constraint for rows > cols
                M = M.T
            P = torch.mm(M, M.T)
            PP = torch.mm(P, P.T)
            trace_P = torch.trace(P) #trace_P is the sum of eigenvalues of P
            trace_PP = torch.trace(PP)#trace_P_P is the sum-square of eigenvalues of P (f)
            ratio = trace_PP * P.shape[0] / (trace_P * trace_P)

            # the following is the tweak to avoid divergence (more info in Kaldi)
            assert ratio > 0.99
            if ratio > 1.02:
                update_speed *= 0.5
                if ratio > 1.1:
                    update_speed *= 0.5

            scale2 = trace_PP/trace_P
            update = P - (torch.matrix_power(P, 0) * scale2)
            alpha = update_speed / scale2
            update = (-4.0 * alpha) * torch.mm(update, M)
            updated = M + update
            # updated has shape (cols, rows) if rows > cols, else has shape (rows, cols)
            # Transpose (or not) to shape (cols, rows) (IMPORTANT, s.t. correct dimensions are reshaped)
            # Then reshape to (cols, in_filters, kernel_width)
            return updated.reshape(*orig_shape) if mshape[0] > mshape[1] else updated.T.reshape(*orig_shape)

    @staticmethod
    def get_M_shape(conv_weight):
        orig_shape = conv_weight.shape #(out_channels, in_channels, kernel_size)
        return (orig_shape[1]*orig_shape[2], orig_shape[0]) #Transformed shape (M) = (64 × 3, 128) → (192, 128)


    @staticmethod
    def get_semi_orth_error(conv1dlayer):
        with torch.no_grad():
            orig_shape = conv1dlayer.weight.shape
            M = conv1dlayer.weight.reshape(
                orig_shape[0], orig_shape[1]*orig_shape[2]).T
            mshape = M.shape
            if mshape[0] > mshape[1]:  # semi orthogonal constraint for rows > cols
                M = M.T
            P = torch.mm(M, M.T)
            PP = torch.mm(P, P.T)
            trace_P = torch.trace(P)
            trace_PP = torch.trace(PP)
            scale2 = torch.sqrt(trace_PP/trace_P) ** 2
            update = P - (torch.matrix_power(P, 0) * scale2)
            return torch.norm(update, p='fro')

#Continuous scaled dropout
class SharedDimScaleDropout(nn.Module):
    def __init__(self, alpha: float = 0.5, dim=1):
        '''
        Continuous scaled dropout that is const over chosen dim (usually across time)
        Multiplies inputs by random mask taken from Uniform([1 - 2\alpha, 1 + 2\alpha])

        For time-series or sequence models (e.g., RNNs, Transformers), the mask is constant along the time dimension, preserving temporal structure.
        '''
        super(SharedDimScaleDropout, self).__init__()
        if alpha > 0.5 or alpha < 0:
            raise ValueError("alpha must be between 0 and 0.5")
        self.alpha = alpha
        self.dim = dim
        self.register_buffer('mask', torch.tensor(0.)) #method used to register a tensor as part of the model without considering it as a trainable parameter

    def forward(self, X):
        if self.training:
            if self.alpha != 0.:
                # sample mask from uniform dist with dim of length 1 in self.dim and then repeat to match size
                tied_mask_shape = list(X.shape)
                tied_mask_shape[self.dim] = 1
                repeats = [1 if i != self.dim else X.shape[self.dim]
                           for i in range(len(X.shape))]
                return X * self.mask.repeat(tied_mask_shape).uniform_(1 - 2*self.alpha, 1 + 2*self.alpha).repeat(repeats)
                # expected value of dropout mask is 1 so no need to scale outputs like vanilla dropout
        return X

#Cristal CLear
class FTDNNLayer(nn.Module):

    def __init__(self, in_dim, out_dim, bottleneck_dim, context_size=2, dilations=None, paddings=None, alpha=0.0):
        '''
        3 stage factorised TDNN http://danielpovey.com/files/2018_interspeech_tdnnf.pdf
        '''
        """Yes, this code implements the concept of factorizing the convolution as described in the paper.
          It breaks down a potentially large convolution (like a 3x1 convolution) into two smaller convolutions (2x1),
            with the first two layers being constrained by the semi-orthogonal property. The third layer is a regular convolution
            that maps to the final output dimension.
            {The setup described above uses a constrained
            3x1 convolution followed by 1x1 convolution (or equivalently, a TDNN
            layer splicing 3 frames followed immediately by a feedforward
            layer). We have found that better results can be obtained by
            using a constrained 2x1 convolution followed by a 2x1 convolution. We refer to this as “factorizing the convolution”.}

            However here we use 3 stage sliceing :3-stage splicing
            {Something that we have found to be even better than the “factorized convolution” above, is to have a layer with a constrained
            2x1 convolution to dimension 256, followed by another constrained 2x1 convolution to dimension 256, followed by a 2x1
            convolution back to the hidden-layer dimension (e.g. 1280).
            The dimension now goes from, for example, 1280 → 256 →
            256 → 1280, within one layer. The effective temporal context
            of this setup is of course wider than the TDNN baseline, due to
            the extra 2x1 convolution}
"""
        super(FTDNNLayer, self).__init__()
        paddings = [1, 1, 1] if not paddings else paddings
        dilations = [2, 2, 2] if not dilations else dilations
        assert len(paddings) == 3
        assert len(dilations) == 3
        self.factor1 = SOrthConv(in_dim, bottleneck_dim, context_size, 
                                 padding=paddings[0], dilation=dilations[0])
        self.factor2 = SOrthConv(bottleneck_dim, bottleneck_dim,
                                 context_size, padding=paddings[1], dilation=dilations[1]) #we used Bottelneck to reduce feature space and improve effeciency ==> Why use it? → Fewer parameters, better generalization, faster inference.
        self.factor3 = nn.Conv1d(bottleneck_dim, out_dim, context_size,
                                 padding=paddings[2], dilation=dilations[2], bias=False)
        self.nl = nn.ReLU()
        self.bn = nn.BatchNorm1d(out_dim)
        self.dropout = SharedDimScaleDropout(alpha=alpha, dim=1)

    def forward(self, x):
        ''' input (batch_size, seq_len, in_dim) '''
        assert (x.shape[-1] == self.factor1.conv.weight.shape[1])
        x = self.factor1(x.transpose(1, 2))
        x = self.factor2(x)
        x = self.factor3(x)
        x = self.nl(x)
        x = self.bn(x).transpose(1, 2) #batchNorm1 expects a  (batch_size, num_channels, seq_len) format
        x = self.dropout(x)
        return x

    def step_semi_orth(self):
        for layer in self.children():
            if isinstance(layer, SOrthConv):
                layer.step_semi_orth()

    def orth_error(self):
        orth_error = 0
        for layer in self.children():
            if isinstance(layer, SOrthConv):
                orth_error += layer.orth_error()
        return orth_error

#Clearr
class DenseReLU(nn.Module): 

    def __init__(self, in_dim, out_dim):
        super(DenseReLU, self).__init__()
        self.fc = nn.Linear(in_dim, out_dim)
        self.bn = nn.BatchNorm1d(out_dim)
        self.nl = nn.ReLU()

    def forward(self, x):
        x = self.fc(x) #expects (batch_size, sequence_length, in_dim).
        x = self.nl(x)
        if len(x.shape) > 2:
            x = self.bn(x.transpose(1, 2)).transpose(1, 2) #expects input of shape (batch_size, features, sequence_length)
        else:
            x = self.bn(x)
        return x

#meth1 :simple pooling layer [CLear] 
class StatsPool(nn.Module):

    def __init__(self, floor=1e-10, bessel=False):
        super(StatsPool, self).__init__()
        self.floor = floor
        self.bessel = bessel

    def forward(self, x):
        #(batch_size, sequence_length, feature_dim)
        means = torch.mean(x, dim=1)
        _, t, _ = x.shape
        if self.bessel:
            t = t - 1
        residuals = x - means.unsqueeze(1)
        numerator = torch.sum(residuals**2, dim=1)
        stds = torch.sqrt(torch.clamp(numerator, min=self.floor)/t)
        x = torch.cat([means, stds], dim=1) #[batch,means*stds]
        return x
    
    
#meth2 : temporal pooling functions with learnable parameters:NEtVlad [CLearrrr]
class NetVLAD(nn.Module):
    def __init__(self, num_clusters=8, dim=128, alpha=100.0, normalize_input=True):
        """
        Args:
            num_clusters (int): Number of clusters (K).
            dim (int): Dimension of the input descriptors.
            alpha (float): Scaling parameter used to sharpen or smooth the soft assignment.
            normalize_input (bool): If True, L2 normalize input descriptors along the feature dimension.
        """
        super(NetVLAD, self).__init__()
        self.num_clusters = num_clusters
        self.dim = dim
        self.alpha = alpha
        self.normalize_input = normalize_input

        # This convolution layer computes the assignment weights for each descriptor to each cluster.
        # It takes input of size [B, dim, N] and outputs a score for each cluster at each time step.
        self.conv = nn.Conv1d(dim, num_clusters, kernel_size=1, bias=True)
        
        # The cluster centers (or "visual words"). This is a trainable parameter of shape [K, dim].
        self.centroids = nn.Parameter(torch.rand(num_clusters, dim))

    def forward(self, x):
        """
        Args:
            x (torch.Tensor): Input tensor of shape [B, dim, N], where:
                B = batch size,
                dim = feature dimension,
                N = number of frames (or audio descriptors).
                
        Returns:
            torch.Tensor: The NetVLAD descriptor for each input in the batch.
                          Output shape is [B, num_clusters * dim].
        """
        # Optionally normalize each descriptor (feature vector) to unit length.
        if self.normalize_input:
            x = F.normalize(x, p=2, dim=1)

        # Compute the assignment scores using the 1x1 convolution.
        # Output shape: [B, num_clusters, N]
        soft_assign = self.conv(x)
        
        # Apply the softmax along the cluster dimension to get soft assignments.
        # We scale the scores with alpha to control the sharpness.
        soft_assign = F.softmax(self.alpha * soft_assign, dim=1)

        # Permute x to shape [B, N, dim] for easier computation of residuals.
        x_flatten = x.permute(0, 2, 1).contiguous()  # Shape: [B, N, dim]

        # Expand the cluster centers to match the batch size.
        # New shape: [B, num_clusters, dim]
        centroids = self.centroids.expand(x.size(0), -1, -1)

        # Compute the residuals between each descriptor and each cluster center.
        # First, expand x_flatten to shape [B, 1, N, dim]
        # Expand centroids to shape [B, num_clusters, 1, dim]
        x_expanded = x_flatten.unsqueeze(1)         # Shape: [B, 1, N, dim]
        centroids_expanded = centroids.unsqueeze(2)   # Shape: [B, num_clusters, 1, dim]
        residuals = x_expanded - centroids_expanded   # Shape: [B, num_clusters, N, dim]
        # Weight the residuals by the soft assignments.
        # soft_assign is [B, num_clusters, N] so unsqueeze the last dimension.
        soft_assign = soft_assign.unsqueeze(-1)      # Shape: [B, num_clusters, N, 1]
        weighted_residuals = soft_assign * residuals   # Shape: [B, num_clusters, N, dim]

        # Aggregate (sum) the weighted residuals over all descriptors (N).
        vlad = weighted_residuals.sum(dim=2)           # Shape: [B, num_clusters, dim]

        # Intra-normalization: normalize each cluster’s descriptor vector.
        vlad = F.normalize(vlad, p=2, dim=2)

        # Flatten the VLAD descriptor into a single vector per sample.
        vlad = vlad.view(x.size(0), -1)                # Shape: [B, num_clusters * dim]

        # L2 normalize the final VLAD vector.
        vlad = F.normalize(vlad, p=2, dim=1)

        return vlad

#Clearr
class AAMSoftmax(nn.Module): 
    def __init__(self, in_features, out_features, s=30.0, m=0.2):
        """
        Args:
            in_features (int): Dimension of the input speaker embedding.
            out_features (int): Number of speaker classes.
            s (float): Scale factor for the logits (commonly set to a value like 30.0).
            m (float): Angular margin (in radians) to be added to the target class.
        """
        super(AAMSoftmax, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s  # scaling factor
        self.m = m  # angular margin

        # Weight matrix for classification, shape: [num_classes, in_features].
        # This acts as the class centers (or prototypes).
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

    def forward(self, x, labels):
        """
        Args:
            x (torch.Tensor): Input embedding of shape [batch_size, in_features].
            labels (torch.LongTensor): Ground truth labels of shape [batch_size].
        Returns:
            torch.Tensor: Scaled logits with the angular margin applied to the target classes.
        """
        # Normalize the speaker embedding vectors and the weight vectors
        x_norm = F.normalize(x, p=2, dim=1)
        weight_norm = F.normalize(self.weight, p=2, dim=1)

        # Compute cosine similarity between embeddings and class centers.
        # Resulting shape: [batch_size, num_classes]
        cosine = F.linear(x_norm, weight_norm)

        # Clamp cosine values to the valid range of arccos, i.e., [-1, 1], to ensure numerical stability.
        cosine = cosine.clamp(-1, 1)
        # Compute the angle (theta) between x and each class center.
        theta = torch.acos(cosine)  

        # For the target classes, add the angular margin m.
        # This effectively shifts the angle for the correct class by m radians.
        theta_margin = theta + self.m
        # Compute cos(theta + m) using the cosine addition formula:
        # cos(theta + m) = cos(theta)*cos(m) - sin(theta)*sin(m)
        cosine_margin = torch.cos(theta_margin)

        # Create one-hot encoding for the labels.
        one_hot = torch.zeros_like(cosine)
        one_hot.scatter_(1, labels.view(-1, 1), 1.0)

        # For target classes, replace cosine with cosine_margin.
        # For non-target classes, keep the original cosine.
        logits = self.s * (one_hot * cosine_margin + (1.0 - one_hot) * cosine)

        return logits

#CLear as a whistle tdnn simple layer
class TDNN(nn.Module): 

    def __init__(
        self,
        input_dim=23,
        output_dim=512,
        context_size=5,
        stride=1,
        dilation=1,
        batch_norm=True,
        dropout_p=0.0,
        padding=0
    ):
        super(TDNN, self).__init__()
        self.context_size = context_size
        self.stride = stride
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.dilation = dilation
        self.dropout_p = dropout_p
        self.padding = padding

        self.conv = nn.Conv1d(self.input_dim,
                                self.output_dim,
                                self.context_size,
                                stride=self.stride,
                                padding=self.padding,
                                dilation=self.dilation)

        self.nonlinearity = nn.ReLU()
        self.batch_norm = batch_norm
        if batch_norm:
            self.bn = nn.BatchNorm1d(output_dim)
        self.drop = nn.Dropout(p=self.dropout_p)

    def forward(self, x):
        '''
        input: size (batch, seq_len, input_features)
        outpu: size (batch, new_seq_len, output_features)
        '''

        _, _, d = x.shape
        assert (d == self.input_dim), 'Input dimension was wrong. Expected ({}), got ({})'.format(
            self.input_dim, d)

        x = self.conv(x.transpose(1, 2)) # (batch_size, input_dim, sequence_length)
        x = self.nonlinearity(x)
        x = self.drop(x)

        if self.batch_norm:
            x = self.bn(x)
        return x.transpose(1, 2) # Back to (batch_size, sequence_length, output_dim)

#CLear as a whistle just need to check the last methods 
class XEmbedding(nn.Module):

    def __init__(self, in_dim=30):
        '''
        The FTDNN architecture from
        "State-of-the-art speaker recognition with neural network embeddings in 
        NIST SRE18 and Speakers in the Wild evaluations"
        https://www.sciencedirect.com/science/article/pii/S0885230819302700
        '''
        super(XEmbedding, self).__init__()

        self.layer01 = TDNN(input_dim=in_dim, output_dim=512,
                            context_size=5, padding=2)
        self.layer02 = FTDNNLayer(512, 1024, 256, context_size=2, dilations=[
                                  2, 2, 2], paddings=[1, 1, 1])
        self.layer03 = FTDNNLayer(1024, 1024, 256, context_size=1, dilations=[
                                  1, 1, 1], paddings=[0, 0, 0])
        self.layer04 = FTDNNLayer(1024, 1024, 256, context_size=2, dilations=[
                                  3, 3, 2], paddings=[2, 1, 1])
        self.layer05 = FTDNNLayer(2048, 1024, 256, context_size=1, dilations=[
                                  1, 1, 1], paddings=[0, 0, 0])
        self.layer06 = FTDNNLayer(1024, 1024, 256, context_size=2, dilations=[
                                  3, 3, 2], paddings=[2, 1, 1])
        self.layer07 = FTDNNLayer(3072, 1024, 256, context_size=2, dilations=[
                                  3, 3, 2], paddings=[2, 1, 1])
        self.layer08 = FTDNNLayer(1024, 1024, 256, context_size=2, dilations=[
                                  3, 3, 2], paddings=[2, 1, 1])
        self.layer09 = FTDNNLayer(3072, 1024, 256, context_size=1, dilations=[
                                  1, 1, 1], paddings=[0, 0, 0])
        self.layer10 = DenseReLU(1024, 2048)

        self.layer11 = StatsPool()

        self.layer12 = DenseReLU(4096, 512)

        self.layer13=AAMSoftmax(in_features=512, out_features=17, s=30.0, m=0.2)


    def forward(self, x,labels=None):
        '''
        Input must be (batch_size, seq_len, in_dim)
        '''
        x = self.layer01(x)
        x_2 = self.layer02(x)
        x_3 = self.layer03(x_2)
        x_4 = self.layer04(x_3)
        skip_5 = torch.cat([x_4, x_3], dim=-1)
        x = self.layer05(skip_5)
        x_6 = self.layer06(x)
        skip_7 = torch.cat([x_6, x_4, x_2], dim=-1)
        x = self.layer07(skip_7)
        x_8 = self.layer08(x)
        skip_9 = torch.cat([x_8, x_6, x_4], dim=-1)
        x = self.layer09(skip_9)
        x = self.layer10(x)
        x = self.layer11(x)
        x = self.layer12(x)
        if labels is not None:
            logits = self.layer13(x, labels)
            return logits
        else:
        # During inference, compute logits and return predicted class
            logits = self.layer13(x, torch.zeros(x.size(0), dtype=torch.long, device=x.device))
            probabilities = F.softmax(logits, dim=1)  # Shape: [batch_size, num_classes]
            predicted_class = torch.argmax(probabilities, dim=1)  # Shape: [batch_size]
            return predicted_class
        # return x

    def step_ftdnn_layers(self):
        for layer in self.children():
            if isinstance(layer, FTDNNLayer):
                layer.step_semi_orth()

    def set_dropout_alpha(self, alpha):
        for layer in self.children():
            if isinstance(layer, FTDNNLayer):
                layer.dropout.alpha = alpha

    def get_orth_errors(self):
        errors = 0.
        with torch.no_grad():
            for layer in self.children():
                if isinstance(layer, FTDNNLayer):
                    errors += layer.orth_error()
        return errors

In [ ]:
tdnn_f = FTDNNLayer(1280, 512, 256, context_size=2, dilations=[2,2,2], paddings=[1,1,1])
# This is a sequence of three 2x1 convolutions
# dimensions go from 1280 -> 256 -> 256 -> 512
# dilations and paddings handles how much to dilate and pad each convolution
# Having these configurable is to ensure the sequence length stays the same

test_input = torch.rand(5, 100, 1280)
# inputs to the FTDNNLayer must be (batch_size, seq_len, in_dim)

tdnn_f(test_input).shape # returns (5, 100, 512)

tdnn_f.step_semi_orth() # The key method to constrain the first two convolutions, perform after every SGD step

tdnn_f.orth_error() # This returns the orth error of the constrained convs, useful for debugging

In [ ]:
void ConstrainOrthonormalInternal(BaseFloat scale,
                                  const std::string &component_name,
                                  CuMatrixBase<BaseFloat> *M) {
  KALDI_ASSERT(scale != 0.0);

  // We'd like to enforce the rows of M to be orthonormal.
  // define P = M M^T.  If P is unit then M has orthonormal rows.
  // We actually want P to equal scale^2 * I, so that M's rows are
  // orthogonal with 2-norms equal to 'scale'.
  // We (notionally) add to the objective function, the value
  // -alpha times the sum of squared elements of Q = (P - scale^2 * I).
  int32 rows = M->NumRows(), cols = M->NumCols();
  CuMatrix<BaseFloat> M_update(rows, cols);
  CuMatrix<BaseFloat> P(rows, rows);
  P.SymAddMat2(1.0, *M, kNoTrans, 0.0);
  P.CopyLowerToUpper();

  // The 'update_speed' is a constant that determines how fast we approach a
  // matrix with the desired properties (larger -> faster).  Larger values will
  // update faster but will be more prone to instability.  0.125 (1/8) is the
  // value that gives us the fastest possible convergence when we are already
  // close to be a semi-orthogonal matrix (in fact, it will lead to quadratic
  // convergence).
  // See  http://www.danielpovey.com/files/2018_interspeech_tdnnf.pdf
  // for more details.
  BaseFloat update_speed = 0.125;
  bool floating_scale = (scale < 0.0);


  if (floating_scale) {
    // This (letting the scale "float") is described in Sec. 2.3 of
    // http://www.danielpovey.com/files/2018_interspeech_tdnnf.pdf,
    // where 'scale' here is written 'alpha' in the paper.
    //
    // We pick the scale that will give us an update to M that is
    // orthogonal to M (viewed as a vector): i.e., if we're doing
    // an update M := M + X, then we want to have tr(M X^T) == 0.
    // The following formula is what gives us that.
    // With P = M M^T, our update formula is doing to be:
    //  M := M + (-4 * alpha * (P - scale^2 I) * M).
    // (The math below explains this update formula; for now, it's
    // best to view it as an established fact).
    // So X (the change in M) is -4 * alpha * (P - scale^2 I) * M,
    // where alpha == update_speed / scale^2.
    // We want tr(M X^T) == 0.  First, forget the -4*alpha, because
    // we don't care about constant factors.  So we want:
    //  tr(M * M^T * (P - scale^2 I)) == 0.
    // Since M M^T == P, that means:
    //  tr(P^2 - scale^2 P) == 0,
    // or scale^2 = tr(P^2) / tr(P).
    // Note: P is symmetric so it doesn't matter whether we use tr(P P) or
    // tr(P^T P); we use tr(P^T P) because I believe it's faster to compute.

    BaseFloat trace_P = P.Trace(), trace_P_P = TraceMatMat(P, P, kTrans);

    if (trace_P < 1.0e-15)
      return;   // This matrix has almost zero value.  It can happen when
                // components are unused.

    scale = std::sqrt(trace_P_P / trace_P);

    // The following is a tweak to avoid divergence when the eigenvalues aren't
    // close to being the same.  trace_P is the sum of eigenvalues of P, and
    // trace_P_P is the sum-square of eigenvalues of P.  Treat trace_P as a sum
    // of positive values, and trace_P_P as their sumsq.  Then mean = trace_P /
    // dim, and trace_P_P cannot be less than dim * (trace_P / dim)^2,
    // i.e. trace_P_P >= trace_P^2 / dim.  If ratio = trace_P_P * dim /
    // trace_P^2, then ratio >= 1.0, and the excess above 1.0 is a measure of
    // how far we are from convergence.  If we're far from convergence, we make
    // the learning rate slower to reduce the risk of divergence, since the
    // update may not be stable for starting points far from equilibrium.
    BaseFloat ratio = (trace_P_P * P.NumRows() / (trace_P * trace_P));
    if (!(ratio > 0.99)) {
      KALDI_WARN << "Ratio is " << ratio << " (should be >= 1.0); component is "
                 << component_name;
      KALDI_ASSERT(ratio > 0.9);
    }
    if (ratio > 1.02) {
      KALDI_WARN << "Ratio is " << ratio << ", multiplying update speed "
                 << "(currently " << update_speed << ") by 0.5; component is "
                 << component_name;
      update_speed *= 0.5;  // Slow down the update speed to reduce the risk of divergence.
      if (ratio > 1.1) update_speed *= 0.5;  // Slow it down even more.
    }
  }

  P.AddToDiag(-1.0 * scale * scale);

  // We may want to un-comment the following code block later on if we have a
  // problem with instability in setups with a non-floating orthonormal
  // constraint.
  /*
  if (!floating_scale) {
    // This is analogous to the stuff with 'ratio' above, but when we don't have
    // a floating scale.  It reduces the chances of divergence when we have
    // a bad initialization.
    BaseFloat error = P.FrobeniusNorm(),
        error_proportion = error * error / P.NumRows();
    // 'error_proportion' is the sumsq of elements in (P - I) divided by the
    // sumsq of elements of I.  It should be much less than one (i.e. close to
    // zero) if the error is small.
    if (error_proportion > 0.02) {
      update_speed *= 0.5;
      if (error_proportion > 0.1)
        update_speed *= 0.5;
    }
  }
  */

  if (GetVerboseLevel() >= 1) {
    BaseFloat error = P.FrobeniusNorm();
    KALDI_VLOG(2) << "Error in orthogonality is " << error;
  }

  // see Sec. 2.2 of http://www.danielpovey.com/files/2018_interspeech_tdnnf.pdf
  // for explanation of the 1/(scale*scale) factor, but there is a difference in
  // notation; 'scale' here corresponds to 'alpha' in the paper, and
  // 'update_speed' corresponds to 'nu' in the paper.
  BaseFloat alpha = update_speed / (scale * scale);

  // At this point, the matrix P contains what, in the math, would be Q =
  // P-scale^2*I.  The derivative of the objective function w.r.t. an element q(i,j)
  // of Q is now equal to -2*alpha*q(i,j), i.e. we could write q_deriv(i,j)
  // = -2*alpha*q(i,j) This is also the derivative of the objective function
  // w.r.t. p(i,j): i.e. p_deriv(i,j) = -2*alpha*q(i,j).
  // Suppose we have define this matrix as 'P_deriv'.
  // The derivative of the objective w.r.t M equals
  // 2 * P_deriv * M, which equals -4*alpha*(P-scale^2*I)*M.
  // (Currently the matrix P contains what, in the math, is P-scale^2*I).
  M_update.AddMatMat(-4.0 * alpha, P, kNoTrans, *M, kNoTrans, 0.0);
  M->AddMat(1.0, M_update);
}

/**
   This function, to be called after processing every minibatch, is responsible
   for enforcing the orthogonality constraint for any components of type
   LinearComponent or inheriting from AffineComponent that have the
   "orthonormal_constraint" value set.
 */

In [ ]:
model = XEmbedding(in_dim=30) #we will try 0 (HLTCOE) or 40 MFCC (CLSP-MIT) for 16 KHz
summary(model, input_size=(667, 30))  # Adjust input_size to     match (batch_size, input_dim),

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1             [-1, 128, 368]          36,736
       BatchNorm1d-2             [-1, 128, 368]             256
              ReLU-3             [-1, 128, 368]               0
            Conv1d-4              [-1, 64, 368]           8,256
       BatchNorm1d-5              [-1, 64, 368]             128
            Conv1d-6             [-1, 128, 368]           1,792
            Conv1d-7              [-1, 64, 368]           8,256
 AdaptiveAvgPool1d-8                [-1, 64, 1]               0
            Linear-9                    [-1, 4]             256
             ReLU-10                    [-1, 4]               0
           Linear-11                   [-1, 64]             256
          Sigmoid-12                   [-1, 64]               0
          SEBlock-13              [-1, 64, 368]               0
          TCSConv-14              [-1, 

Train Model 

In [ ]:

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, patience=5, grad_clip=1.0):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using..",device)
    model.to(device)

    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=patience, verbose=True)
   # scheduler = OneCycleLR(
    #optimizer,
  #  max_lr=0.01,  # Maximum learning rate
   # steps_per_epoch=steps_per_epoch,
  #  epochs=num_epochs,
   # pct_start=0.3,  # Percentage of the cycle to increase learning rate
   # anneal_strategy='cos',  # Annealing strategy ('linear' or 'cos')
   # div_factor=25.0,  # Initial learning rate = max_lr / div_factor
   # final_div_factor=1e4  # Minimum learning rate = initial_lr / final_div_factor
#)

    best_val_loss = float('inf')
    for epoch in range(100):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            inputs = inputs.permute(0, 2, 1)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()

            # Gradient clipping
            #nn.utils.clip_grad_norm_(model.parameters(), grad_clip) #maybe

            optimizer.step()
            if (epoch+1)%4==0:
                model.step_semi_orth() # The key method to constrain the first two convolutions, perform after every 4 SGD step
                with open("orth_error_log.txt", "w") as f:
                    f.write(str(model.get_orth_errors())) # This returns the orth error of the constrained convs, useful for debugging
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()

        train_loss /= len(train_loader)
        train_acc = train_correct / train_total

        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                inputs = inputs.permute(0, 2, 1)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()

        val_loss /= len(val_loader)
        val_acc = val_correct / val_total

        print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        print("--------------------")

        # Adjust learning rate based on validation loss
        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'XEmbedding/XEmbedding_101.pt')

    print("Training completed.")

num_classes = len(train_dataset.label_encoder.classes_)
model = XEmbedding()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=100)

Using.. cpu
Epoch [1/100] Train Loss: 1.5863, Train Acc: 0.5841 Val Loss: 1.0931, Val Acc: 0.7338
--------------------
Epoch [2/100] Train Loss: 0.7238, Train Acc: 0.8242 Val Loss: 0.6187, Val Acc: 0.8331
--------------------
Epoch [3/100] Train Loss: 0.4831, Train Acc: 0.8760 Val Loss: 0.6623, Val Acc: 0.8075
--------------------
Epoch [4/100] Train Loss: 0.3820, Train Acc: 0.8974 Val Loss: 0.5425, Val Acc: 0.8466
--------------------
Epoch [5/100] Train Loss: 0.3376, Train Acc: 0.9008 Val Loss: 0.3562, Val Acc: 0.8932
--------------------
Epoch [6/100] Train Loss: 0.2881, Train Acc: 0.9146 Val Loss: 0.4443, Val Acc: 0.8617
--------------------
Epoch [7/100] Train Loss: 0.2390, Train Acc: 0.9295 Val Loss: 0.3815, Val Acc: 0.8887
--------------------
Epoch [8/100] Train Loss: 0.2284, Train Acc: 0.9303 Val Loss: 0.3172, Val Acc: 0.8962
--------------------
Epoch [9/100] Train Loss: 0.2164, Train Acc: 0.9365 Val Loss: 0.3615, Val Acc: 0.8707
--------------------
Epoch [10/100] Train Loss

Inference

In [13]:
test_dataset = OptimizedAudioDataset(csv_file="test.csv",
                                      audio_dir="test/Test",
                                      transform=optimized_audio_transform,
                                      is_test=True)

test_loader = DataLoader(test_dataset, batch_size=70, shuffle=False,
                          collate_fn=custom_collate_fn, num_workers=0)
                        
def predict_test_data(model, test_loader):
    device = "cpu"
    model.to(device)
    print("MODEL is using CPU for Inferencing....")
    model.eval()
    predictions = []

    with torch.inference_mode():
        for inputs in tqdm(test_loader, desc="Inferencing"):
            inputs = inputs.to(device)
            inputs = inputs.permute(0, 2, 1)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            predictions.extend(predicted.cpu().numpy())

    return predictions

test_predictions = predict_test_data(model_inference, test_loader)

Preloading audio: 100%|██████████| 7/7 [00:00<00:00, 456.40it/s]


MODEL is using CPU for Inferencing....


Inferencing: 100%|██████████| 1/1 [00:00<00:00,  4.97it/s]


In [14]:
test=pd.read_csv("test.csv")
test.head(8)

,path,label
0,C:\CNN Project\test\Test\Ahmed_ali_ajmi.wav,Ahmed_ali_ajmi.m4a
1,C:\CNN Project\test\Test\Alimahmoud.wav,Alimahmoud.m4a
2,C:\CNN Project\test\Test\Islem_sob7i.wav,Islem_sob7i.m4a
3,C:\CNN Project\test\Test\Yaser_aldousari.wav,Yaser_aldousari.m4a
4,C:\CNN Project\test\Test\Youssef_aidours.wav,Youssef_aidours.m4a
5,C:\CNN Project\test\Test\phone_audio.wav,yasser
6,C:\CNN Project\test\Test\surah_al_mulk-yasser_...,yasser


In [15]:
train_dataset.label_encoder.inverse_transform(test_predictions)


array(['ahmed_alijmi', 'abd_albassit_abd_samad', 'abd_rahman_sodays',
       'abd_albassit_abd_samad', 'abd_rahman_sodays', 'yasser_aldossari',
       'yasser_aldossari'], dtype=object)